# 🌍 Analyse de vitesses de vent – Données Stations vs OpenMeteo
Ce notebook vous permet de :
- Charger une liste de sites avec coordonnées GPS
- Identifier les stations météo les plus proches
- Récupérer les données historiques de vent (70 ans si dispo)
- Comparer avec les données issues de l'API OpenMeteo
- Exporter les résultats en fichiers CSV

In [ ]:
import pandas as pd
from meteostat import Stations, Daily, Point
from datetime import datetime
import matplotlib.pyplot as plt
import requests

In [ ]:
# 🔧 Fonctions de récupération des données

def get_station_data(lat, lon, start, end):
    point = Point(lat, lon)
    stations = Stations().nearby(lat, lon).fetch(5)

    print(f"Stations proches ({lat}, {lon}) :")
    print(stations[["name", "id", "distance", "inventory"]])

    selected_station_id = stations.index[0]  # à personnaliser si besoin
    data = Daily(selected_station_id, start, end)
    df = data.fetch()
    df.reset_index(inplace=True)
    df = df[["time", "wspd"]].dropna()
    return df

def get_openmeteo_wind(lat, lon, start_date, end_date):
    url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}&start_date={start_date}&end_date={end_date}"
        f"&daily=windspeed_10m_max&timezone=UTC"
    )
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame({
        "time": data["daily"]["time"],
        "wspd": data["daily"]["windspeed_10m_max"]
    })
    return df

In [ ]:
# 📥 Chargement du fichier CSV modèle

sites = pd.read_csv("modele_sites.csv", sep=";")
sites.columns = sites.columns.str.strip()
print(sites)

In [ ]:
# 🔄 Traitement de chaque site

start = datetime(1953, 1, 1)
end = datetime(2023, 12, 31)

for _, row in sites.iterrows():
    name = row["Nom"]
    lat = row["Latitude"]
    lon = row["Longitude"]
    print(f"\n🔍 Traitement du site : {name} ({lat}, {lon})")

    obs_data = get_station_data(lat, lon, start, end)
    obs_data.to_csv(f"{name}_station.csv", index=False)

    api_data = get_openmeteo_wind(lat, lon, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
    api_data.to_csv(f"{name}_openmeteo.csv", index=False)

    # 📊 Affichage comparatif
    plt.figure(figsize=(12, 4))
    plt.plot(obs_data["time"], obs_data["wspd"], label="Station Meteostat", alpha=0.7)
    plt.plot(api_data["time"], api_data["wspd"], label="OpenMeteo API", alpha=0.7)
    plt.title(f"Comparaison vitesses du vent – {name}")
    plt.xlabel("Date")
    plt.ylabel("Vitesse (m/s)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()